In [8]:
import sys
import os

# find and append the root directory
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))  # or ".." if in notebooks/
sys.path.append(project_root)

import numpy as np
import pandas as pd
import itertools
from tqdm import tqdm
import time

from src.problems.benchmarks import (
    rosenbrock, rastrigin, ackley,
    grad_rosenbrock, grad_rastrigin, grad_ackley
)
from src.optimizers.sa import sa_continuous
from src.optimizers.gd import gradient_descent
from src.utils.utils_experiments import bootstrap_experiment_benchmarks

# Paths
results_dir = os.path.join(project_root, "results", "gridsearch")
os.makedirs(results_dir, exist_ok=True)

In [9]:
benchmarks = {
    "rosenbrock": (rosenbrock, grad_rosenbrock),
    "rastrigin": (rastrigin, grad_rastrigin),
    "ackley": (ackley, grad_ackley)
}

In [14]:
# hyperparameter grids for gridsearch
sa_grid = {
    'T0': [10, 50, 100],
    'alpha': [0.9, 0.95, 0.99, 0.995, 0.999],
    'step_size': [0.005, 0.01, 0.05, 0.1, 0.3, 0.5]
}

gd_grid = {
    'lr': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.3, 0.5]
}

In [15]:
# SA gridsearch helpers
def run_one_sa(f, benchmark_name, T0, alpha, step_size, num_runs=25):
    result = bootstrap_experiment_benchmarks(
        sa_continuous,
        runs=num_runs,
        f=f,
        dim=2,
        name=benchmark_name,
        T0=T0, alpha=alpha, step_size=step_size,
        tol=1e-6, max_iter=20000
    )
    stats = result["stats"]
    return {
        "T0": T0, "alpha": alpha, "step_size": step_size,
        "mean": stats["mean"], "best": stats["best"], "worst": stats["worst"], "std": stats["std"],
        "epsilon": stats["epsilon"], "near_optimal_count": stats["near_optimal_count"]
    }


def grid_search_sa(f, grid, num_runs=25, benchmark_name=""):
    combos = list(itertools.product(grid["T0"], grid["alpha"], grid["step_size"]))
    total = len(combos)
    results = []

    for idx, (T0, alpha, step_size) in enumerate(combos, 1):
        print(f"[SA | {benchmark_name}] Combo {idx}/{total} — Running...")
        res = run_one_sa(f, benchmark_name, T0, alpha, step_size, num_runs)
        results.append(res)

    return pd.DataFrame(results)


In [16]:
# GD grid search helpers
def run_one_gd(f, grad, benchmark_name, lr, num_runs=25):
    result = bootstrap_experiment_benchmarks(
        gradient_descent,
        runs=num_runs,
        f=f,
        grad_f=grad,
        dim=2,
        name=benchmark_name,
        lr=lr, tol=1e-6, max_iter=20000
    )
    stats = result["stats"]
    return {
        "lr": lr,
        "mean": stats["mean"], "best": stats["best"], "worst": stats["worst"], "std": stats["std"],
        "epsilon": stats["epsilon"], "near_optimal_count": stats["near_optimal_count"]
    }

def grid_search_gd(f, grad, grid, num_runs=25, benchmark_name=""):
    combos = list(grid["lr"])
    total = len(combos)
    results = []

    for idx, lr in enumerate(combos, 1):
        print(f"[GD | {benchmark_name}] Combo {idx}/{total} — Running...")
        res = run_one_gd(f, grad, benchmark_name, lr, num_runs)
        results.append(res)

    return pd.DataFrame(results)


# Running actual gridsearch

In [ ]:
# run the grid search and save results
for name, (f, grad) in benchmarks.items():
    print(f"\nBenchmark: {name}")

    print("Running GD grid search")
    df_gd = grid_search_gd(f, grad, gd_grid, num_runs=25, benchmark_name=name)
    df_gd.to_csv(os.path.join(results_dir, f"gridsearch_gd_{name}_new.csv"), index=False)

    print("Running SA grid search")
    df_sa = grid_search_sa(f, sa_grid, num_runs=25, benchmark_name=name)
    df_sa.to_csv(os.path.join(results_dir, f"gridsearch_sa_{name}_new.csv"), index=False)


Benchmark: rosenbrock
Running GD grid search
[GD | rosenbrock] Combo 1/7 — Running...
[Benchmark] Run 1/25
Clip range : (-2, 2)
[Benchmark] Run 2/25
Clip range : (-2, 2)
[Benchmark] Run 3/25
Clip range : (-2, 2)
[Benchmark] Run 4/25
Clip range : (-2, 2)
[Benchmark] Run 5/25
Clip range : (-2, 2)
[Benchmark] Run 6/25
Clip range : (-2, 2)
[Benchmark] Run 7/25
Clip range : (-2, 2)
[Benchmark] Run 8/25
Clip range : (-2, 2)
[Benchmark] Run 9/25
Clip range : (-2, 2)
[Benchmark] Run 10/25
Clip range : (-2, 2)
[Benchmark] Run 11/25
Clip range : (-2, 2)
[Benchmark] Run 12/25
Clip range : (-2, 2)
[Benchmark] Run 13/25
Clip range : (-2, 2)
[Benchmark] Run 14/25
Clip range : (-2, 2)
[Benchmark] Run 15/25
Clip range : (-2, 2)
[Benchmark] Run 16/25
Clip range : (-2, 2)
[Benchmark] Run 17/25
Clip range : (-2, 2)
[Benchmark] Run 18/25
Clip range : (-2, 2)
[Benchmark] Run 19/25
Clip range : (-2, 2)
[Benchmark] Run 20/25
Clip range : (-2, 2)
[Benchmark] Run 21/25
Clip range : (-2, 2)
[Benchmark] Run 22/